<a href="https://colab.research.google.com/github/vishakhun/EU-AI-ACT-Chatbot/blob/main/Chat_with_EU_AI_ACT_Mistral7B_llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chat with EU AI ACT - Mistral7B and llamaindex
This Colab notebook showcases the integration of advanced AI models for document processing and information retrieval, focusing on the EU AI Act dataset. Key tools and libraries used include:

* Llama CPP and Langchain Embeddings: For advanced text processing and semantic analysis.
* llama-index: To efficiently index and retrieve document data.
* Transformers and Sentence-Transformers: For accessing pre-trained language models and embeddings.
* Python Utilities: Such as logging and sys for effective logging and system operations.

The notebook demonstrates the setup, configuration, and application of these technologies in creating a query engine capable of handling complex document-based queries.




#1: Environment Setup
## 1.1 Installing Required Libraries

In [ ]:
%%capture

!pip install -q pypdf
!pip install -q python-dotenv
!pip install -q transformers
!pip install langchain
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir
!pip install -q llama-index
!pip -q install sentence-transformers

#2: Basic Configurations
## 2.1 Logging Configuration


In [ ]:
import logging
import sys
# Setting up the logging configuration

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

# Section 3: Data Loading and Processing
## 3.1 Loading Documents


In [ ]:
documents = SimpleDirectoryReader("/content/sample_data/EU_AI_ACT").load_data()

#4: Llama CPP Model Setup
## 4.1 Initializing Llama CPP


In [ ]:
import torch

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
# Detailed setup and initialization of the Llama CPP model
    # Model URL and other configurations

llm = LlamaCPP(
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    context_window=3900,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": -1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

#5: Langchain Embeddings Integration
## 5.1 Embedding Model Setup


In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings import LangchainEmbedding
from llama_index import ServiceContext
# Setting up Langchain Embedding model

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="thenlper/gte-large")
)

#6: Service Context and Indexing
## 6.1 Creating Service Context


In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)

### 6.2 Document Indexing


In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

#7: Query Engine and Responses

In [ ]:
# Initializing the query engine

query_engine = index.as_query_engine()
response = query_engine.query("Which AI systems are considered high risk?")

In [ ]:
print(response)

In [ ]:
# User query handling loop

while True:
  query=input()
  response = query_engine.query(query)
  print(response)